与多元线性回归模型和GBDT回归模型相比较，看看XGBoost模型的优越性。

## 案例背景
为了降低不良贷款率，保障自身资金安全，提高风险控制水平，银行等金融机构会根据客户的信用历史资料构建信用评分卡模型给客户评分。根据客户的信用得分，可以预估客户按时还款的可能性，并据此决定是否发放贷款及贷款的额度和利率。

## 一、多元线性回归模型
### 1、读取数据
读取1000条信用卡客户的数据。为了便于演示，这里只选取5个特征变量，分别为客户的月收入、年龄、性别、历史授信额度和历史违约次数，目标变量是客户的信用评分（取值范围为0～100）。

In [40]:
import pandas as pd
df = pd.read_excel('datasets/信用卡评分模型.xlsx')
df.head()

,月收入,年龄,性别,历史授信额度,历史违约次数,信用评分
0,7783,29,0,32274,3,73
1,7836,40,1,6681,4,72
2,6398,25,0,26038,2,74
3,6483,23,1,24584,4,65
4,5167,23,1,6710,3,73


### 2、提取特征变量和目标变量

In [41]:
x = df.drop(columns='信用评分')
y = df['信用评分']

### 3、模型训练及搭建

In [42]:
from sklearn.linear_model import LinearRegression # 从Scikit-Learn库中引入线性回归模型LinearRegression
model = LinearRegression() # 将LinearRegression()赋给变量model
model.fit(x, y) # 用fit()函数进行模型训练

LinearRegression()

### 4、线性回归模型构造

In [43]:
print('各个系数' + str(model.coef_))
print('常数项系数' + str(model.intercept_))

各个系数[ 5.58658996e-04  1.62842002e-01  2.18430276e-01  6.69996665e-05
 -1.51063940e+00]
常数项系数67.16686603853253


多元线性回归方程如下：
$y = 67.16 + 0.000558*x1 + 0.162*x2 + 0.218*x3 + 0.0000669*x4 - 1.51*x5$

### 5、模型评估

In [44]:
import statsmodels.api as sm
x2 = sm.add_constant(x)
est = sm.OLS(y, x2).fit()
print(est.summary())

                            OLS Regression Results                            
Dep. Variable:                   信用评分   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     337.6
Date:                Sun, 09 Oct 2022   Prob (F-statistic):          2.32e-211
Time:                        08:31:14   Log-Likelihood:                -2969.8
No. Observations:                1000   AIC:                             5952.
Df Residuals:                     994   BIC:                             5981.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         67.1669      1.121     59.906      0.0

/Users/dayao/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


可以看到，模型整体的R-squared值为0.629，Adj.R-squared值为0.628，整体拟合效果一般，可能是因为数据量偏少。再来观察P值，可以发现大部分特征变量的P值都较小（小于0.05），的确是和目标变量“信用评分”显著相关的，而特征变量“性别”的P值达到了0.466，说明该特征变量与目标变量没有显著相关性，这也的确符合经验认知。因此，在多元线性回归模型中可以舍去“性别”这一特征变量。

## 二、GBDT回归模型
### 1、读取数据

In [45]:
import pandas as pd
df = pd.read_excel('datasets/信用卡评分模型.xlsx')

### 2、提取特征变量和目标变量

In [46]:
x = df.drop(columns='信用评分')
y = df['信用评分']

### 3、划分训练集和测试集

In [47]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123) # 设置train_test_split()函数的test_size参数为0.2，即测试集数据占20%；设置random_state参数为123（此数字无特殊含义，可以换成其他数字），使得每次运行代码划分数据的结果保持一致

### 4、模型训练及搭建

In [48]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor() # 使用默认参数
model.fit(x_train, y_train)

GradientBoostingRegressor()

### 5、模型预测及评估

In [49]:
y_pred = model.predict(x_test)
y_pred[0:10]

array([70.77631652, 71.40032104, 73.73465155, 84.52533945, 71.09188294,
       84.9327599 , 73.72232388, 83.44560704, 82.61221486, 84.86927209])

汇总预测值和实际值，以便进行对比

In [50]:
a = pd.DataFrame() # 创建一个空DataFrame
a['预测值'] = list(y_pred)
a['实际值'] = list(y_test)
a.head()

,预测值,实际值
0,70.776317,79
1,71.400321,80
2,73.734652,62
3,84.525339,89
4,71.091883,80


GradientBoostingRegressor是一个回归模型，可以通过如下代码查看其R-squared值来评价模型的拟合效果

In [51]:
from sklearn.metrics import r2_score # 从Scikit-Learn库中引入r2_score()函数
r2 = r2_score(y_test, model.predict(x_test)) # 将测试集的实际值和模型的预测值传入r2_score()函数，得出R-squared值
r2
#model.score(x_test, y_test)

0.6742641046430244

得出R-squared值为0.675，这个结果与多元线性回归模型获得的0.629相比是有所改善的.

## 三、XGBoost回归模型
### 1、读取数据

In [60]:
import pandas as pd
df = pd.read_excel('datasets/信用卡评分模型.xlsx')
df.head()

,月收入,年龄,性别,历史授信额度,历史违约次数,信用评分
0,7783,29,0,32274,3,73
1,7836,40,1,6681,4,72
2,6398,25,0,26038,2,74
3,6483,23,1,24584,4,65
4,5167,23,1,6710,3,73


### 2、提取特征变量和目标变量

In [61]:
x = df.drop(columns='信用评分')
y = df['信用评分']

### 3、划分训练集和测试集 

In [62]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123) # 设置train_test_split()函数的test_size参数为0.2，即测试集数据占20%；设置random_state参数为123（此数字无特殊含义，可以换成其他数字），使得每次运行代码划分数据的结果保持一致

### 4、模型训练及搭建

In [63]:
from xgboost import XGBRegressor
model = XGBRegressor() # 使用默认参数
model.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

### 5、模型预测及评估

In [64]:
y_pred = model.predict(x_test)
y_pred[0:10]

array([74.62306 , 69.01495 , 76.393486, 83.88998 , 71.5683  , 86.257324,
       76.0784  , 81.38994 , 81.05504 , 83.24717 ], dtype=float32)

汇总预测值和实际值，以便进行对比

In [65]:
a = pd.DataFrame() # 创建一个空DataFrame
a['预测值'] = list(y_pred)
a['实际值'] = list(y_test)
a.head()

,预测值,实际值
0,74.623062,79
1,69.014954,80
2,76.393486,62
3,83.889977,89
4,71.568298,80


XGBRegressor是一个回归模型，可以通过计算R-squared值来评估模型的拟合效果

In [66]:
from sklearn.metrics import r2_score # 从Scikit-Learn库中引入r2_score()函数
r2 = r2_score(y_test, model.predict(x_test)) # 将测试集的实际值和模型的预测值传入r2_score()函数，得出R-squared值
r2

0.5715437436791975

In [59]:
model.score(x_test, y_test)

0.5715437436791975

### 6、查看特征重要性

In [67]:
features = x.columns # 获取特征名称
importances = model.feature_importances_ # 获取特征重要性
# 整理成二维表格，并按特征重要性降序排列
importances_df = pd.DataFrame()
importances_df['特征名称'] = features
importances_df['特征重要性'] = importances
importances_df.sort_values('特征重要性', ascending=False)

,特征名称,特征重要性
0,月收入,0.324461
4,历史违约次数,0.307467
3,历史授信额度,0.202864
1,年龄,0.098869
2,性别,0.066339


### 7、模型参数调优

In [68]:
from sklearn.model_selection import GridSearchCV
parameters = {'max_depth':[1, 3, 5], 'n_estimators':[50, 100, 150], 'learning_rate':[0.01, 0.05, 0.1, 0.2]}
clf = XGBRegressor()
grid_search = GridSearchCV(model, parameters, scoring='r2', cv=5)

需要注意的是，因为XGBRegressor是回归模型，所以参数调优时应该选择R-squared值作为评估标准，即第4行代码中的scoring参数需要设置成'r2'，而不是分类模型中常用的准确度'accuracy'或ROC曲线的AUC值'roc_auc'。

In [69]:
# 获取参数的最优值
grid_search.fit(x_train, y_train) # 传入训练数据
grid_search.best_params_ # 输出参数的最优值

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}

### 8、用参数的最优值搭建新模型

In [71]:
model = XGBRegressor(max_depth=3, n_estimators=50, learning_rate=0.1)
model.fit(x_train, y_train)

# r2_score()函数进行模型评估 
from sklearn.metrics import r2_score
r2 = r2_score(y_test, model.predict(x_test)) # 也可以用model.score（X_test,y_test）进行评估
r2

0.6884486054771359

获得调优后的R-squared值为0.688，优于调优前的R-squared 0.57

## 补充知识点：数据预处理对于XGBoost模型的必要性
在传统的机器学习模型中，往往需要做很多数据预处理工作，如数据的标准化、缺失值和异常值的处理等。但是对于XGBoost模型而言，很多预处理都不是必要的，例如缺失值，XGBoost模型会自动处理，它会通过列举所有缺失值在当前节点是进入左子树还是右子树来决定缺失值的处理方式。此外，因为XGBoost模型是基于决策树模型的，所以像线性回归等模型需要的特征变换（如离散化、标准化、取log、共线性问题处理）等预处理工作，XGBoost模型都不太需要，这也是树模型的优点之一。进行特征变换后，如数据标准化，会发现最终的结果都是一样的。

绝大部分模型不能自动完成的一步就是特征提取。很多自然语言处理或图像处理的问题，没有现成的特征，需要人工去提取特征。XGBoost模型的确比线性模型要节省很多特征工程的步骤，但特征工程依然是非常必要的，